## Notebook to run retrieval for GMI IWP

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import ipywidgets as w
import matplotlib.pyplot as plt
import numpy as np
import netCDF4
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from torch.utils.data import DataLoader, random_split
from iwc2tb.GMI.gmiSatData import gmiSatData
from iwc2tb.GMI.gmiData import gmiData
from iwc2tb.GMI.GMI_SatData import GMI_Sat
import os
import numpy.ma as ma
import glob
#from iwc2tb.GMI.write_training_data import *
from tqdm.notebook import tqdm
import xarray
import torch

from quantnn.qrnn import QRNN
import time
#time.sleep(60 * 60 * 6)

In [2]:
import sys
sys.path

['/home/inderpreet/git/IWC2TB/notebooks',
 '/home/inderpreet/anaconda3/lib/python37.zip',
 '/home/inderpreet/anaconda3/lib/python3.7',
 '/home/inderpreet/anaconda3/lib/python3.7/lib-dynload',
 '',
 '/home/inderpreet/.local/lib/python3.7/site-packages',
 '/home/inderpreet/anaconda3/lib/python3.7/site-packages',
 '/home/inderpreet/Projects/git/aws',
 '/home/inderpreet/Projects/git/parts',
 '/home/inderpreet/Projects/git/regn',
 '/home/inderpreet/git/pansat',
 '/home/inderpreet/git/IWC2TB',
 '/home/inderpreet/git/quantnn',
 '/home/inderpreet/git/DARDAR_ERA_interpolation',
 '/home/inderpreet/typhon',
 '/home/inderpreet/anaconda3/lib/python3.7/site-packages/locket-0.2.1-py3.7.egg',
 '/home/inderpreet/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/home/inderpreet/.ipython']

## Set inputs

In [3]:
batchSize          = 4
inputs             = np.array(["ta", "t2m",  "wvp", "z0", "stype"])
outputs            = "iwp"

latlims            = [0, 65]
pr                 = None
inputfile          = os.path.expanduser("~/Dendrite/Projects/IWP/GMI/training_data/TB_GMI_train_jan_lpa.nc")
training_data      = gmiData(inputfile, 
                             inputs, 
                             outputs,
                             pratio = pr,
                             batch_size = batchSize,
                             latlims = latlims,
                             )


norm               = training_data.norm


year               = '2017'
month              = '01'


inpath             = os.path.join(os.path.expanduser('~/Dendrite/SatData/GMI/L1B/'), year, month)
gmifiles           = glob.glob(os.path.join(inpath, "*/*.HDF5"))

outpath            = "~/Dendrite/UserAreas/Kaur/IWP/lpa1"

qrnnfile           = "qrnn_gmi_nn_lpa_v0_jan.nc"
qrnn               =  QRNN.load(os.path.join(os.path.expanduser('~/Dendrite/Projects/IWP/GMI/training_data/try_training/')
                                                  , qrnnfile))
quantiles          = qrnn.quantiles
imedian            = np.argwhere((quantiles >= 0.49) & (quantiles < 0.51))[0][0]


{0: (268.379490267815, 20.71525565192133), 1: (264.7705605213822, 23.751568678875426), 2: (265.26470996271524, 14.31355749260509), 3: (256.97768792014574, 9.810320527505446), 4: (286.32800607278887, 14.497505671007154), 5: (22.580422767573513, 16.37695971949476), 6: (163.75704659185553, 451.2063182451618)}


In [4]:
quantiles

array([0.01, 0.03, 0.05, 0.07, 0.09, 0.11, 0.13, 0.15, 0.17, 0.19, 0.21,
       0.23, 0.25, 0.27, 0.29, 0.31, 0.33, 0.35, 0.37, 0.39, 0.41, 0.43,
       0.45, 0.47, 0.49, 0.51, 0.53, 0.55, 0.57, 0.59, 0.61, 0.63, 0.65,
       0.67, 0.69, 0.71, 0.73, 0.75, 0.77, 0.79, 0.81, 0.83, 0.85, 0.87,
       0.89, 0.91, 0.93, 0.95, 0.97, 0.99])

In [5]:
def get_pos_mean(validation_data, qrnn):
    
    quantiles  = qrnn.quantiles
    nquantiles = len(quantiles)
    
    y          = validation_data.y
    y_mean     = np.zeros(validation_data.y.shape)
    y_pre      = np.zeros([validation_data.y.shape[0],validation_data.y.shape[1], len(quantiles)])



    nbatch = len(validation_data)
    ibatch = validation_data.batch_size
    nscan  = validation_data.y.shape[0]
    npix   = validation_data.y.shape[1]
    
    with torch.no_grad():
        for i in range(nbatch):
            
            istart                      = ibatch * i
            iend                        = ibatch * (i + 1) 

            xx, yy                      = validation_data[i]

            xx                          = xx.reshape(-1, validation_data.x.shape[-1])
            y_pred                      = qrnn.predict(xx)
            y_pre[istart:iend, :, :]    = y_pred.reshape(-1, npix, nquantiles).detach().numpy()
            y_mean[istart:iend]         = qrnn.posterior_mean(xx, y_pred).reshape(-1, npix).detach().numpy() 
                   
    return y, y_pre, y_mean

In [6]:
def get_coords(validation_data):
    lat   = validation_data.lat
    lon   = validation_data.lon%360
    t     = validation_data.lst
    stype = validation_data.stype
    tb    = validation_data.x[:, :, :4]
    return tb, lat, lon, stype, t

In [7]:
def predict_iwp(qrnn, filename):


    gmi_s    = GMI_Sat(filename)  

    validation_data    = gmiSatData(gmi_s, 
                             inputs, outputs,
                             batch_size = batchSize,
                             latlims = latlims,
                             normalize = norm,      
                             )
    

    y, y_pre, y_mean = get_pos_mean(validation_data, qrnn)
    
    
    
    tb, lat,  lon, stype, t    = get_coords(validation_data)
    
    return y, y_pre, y_mean, lat, lon, tb, stype, t
    

In [8]:
def plot_iwp(lat, lon, iwp0, iwp,tb, mask):
    
    fig, ax = plt.subplots(1, 2, figsize = [12, 6])
    ax = ax.ravel()
    diff = 100 * (np.exp(np.abs(np.log(iwp/iwp0))) - 1)
    
    bbox = [np.min(lon),np.min(lat),np.max(lon),np.max(lat)] # set bounds for plotting
    n_add = 0
    m = Basemap(llcrnrlon=bbox[0]-n_add,llcrnrlat=bbox[1]-n_add,
                urcrnrlon=bbox[2]+n_add,urcrnrlat=bbox[3]+n_add,resolution='l',
                projection='cyl')
    x, y = m(lon, lat)    
    for var, axes, t in zip([iwp0, iwp], ax, ["GMI", "QRNN"]):
        
        cs = axes.scatter(lon[mask],lat[mask], c = var[mask]* 1000,
                          norm=colors.LogNorm(), vmin = 1, vmax = 10000,)
                        # cmap = cm.Paired)
            
        #cs = m.scatter(lon, lat, var[mask]* 1000, ax = axes)    
        axes.set_title(t)
        ax[0].set_ylabel("Latitude [deg]")
        #axes.set_xlabel("Longitude [deg]")
    cbar = fig.colorbar(cs, ax=[ax[0], ax[1]])
    cbar.ax.set_ylabel("IWP [g/m2]")
      

def get_mask(lat, lon, latlims, lonlims):
    
    im  = (lat >= latlims[0]) & (lat <= latlims[1])
    im1 = (lon >=  lonlims[0]) & (lon < lonlims[1])
    mask  = np.logical_and(im, im1)
    
    return mask    

## Loop over all files

In [9]:
y5 = np.zeros([1, 221])
y95 = np.zeros([1, 221])

for gmifile in tqdm(gmifiles[:]):
    print (gmifile)
    bname = os.path.basename(gmifile)
    outfile = os.path.join(outpath, bname[:-5] + ".nc")
    try:
        y, y_pre, y_pos_mean, lat, lon, tb, stype, t = predict_iwp(qrnn, gmifile)
    except:
        print ("GPROF unavailable")
        continue
        
    y_pre[y_pre < 0] = 0
    y_pos_mean[y_pos_mean < 0] = 0

    y5 = np.concatenate([y5, y_pre[:, :, 2]], axis = 0)
    y95 = np.concatenate([y95, y_pre[:, :, -3]], axis = 0)

    stype = np.argmax(stype, axis = 2)

    print (np.nanmax(y_pos_mean))

    d = xarray.Dataset({
    "iwp": (["scans", "pixels"], y_pre[:, :, imedian]),
    "iwp_mean": (["scans", "pixels"], y_pos_mean),
    "stype": (["scans", "pixels"], stype),
    "local_time": (["scans", "pixels"], t),

    },
    coords={
    "lon": (["scans", "pixels"], lon),
    "lat": (["scans", "pixels"], lat),
    #"quantiles": (["quantiles"], qrnn.quantiles)
    })

    d.to_netcdf(outfile, mode = "w")


        
        
        
    #except:
    #    print ("file not available")
    #    continue

  0%|          | 0/385 [00:00<?, ?it/s]

/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/01/1B.GPM.GMI.TB2016.20170101-S003624-E020857.016155.V05A.HDF5
14.54422378540039
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/01/1B.GPM.GMI.TB2016.20170101-S190715-E203948.016167.V05A.HDF5
15.322071075439453
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/01/1B.GPM.GMI.TB2016.20170101-S221224-E234457.016169.V05A.HDF5
14.00416374206543
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/01/1B.GPM.GMI.TB2016.20170101-S234458-E011731.016170.V05A.HDF5
16.00105094909668
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/01/1B.GPM.GMI.TB2016.20170101-S081915-E095149.016160.V05A.HDF5
12.99700927734375
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/01/1B.GPM.GMI.TB2016.20170101-S203949-E221223.016168.V05A.HDF5
13.487817764282227
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/01/1B.GPM.GMI.TB2016.20170101-S173441-E190714.016166.V05A.HDF5
16.157791137695312
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/01/1B.GPM.GMI.TB2016.20170101-

14.412030220031738
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/05/1B.GPM.GMI.TB2016.20170105-S014818-E032051.016218.V05A.HDF5
14.022887229919434
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/05/1B.GPM.GMI.TB2016.20170105-S032052-E045325.016219.V05A.HDF5
17.074047088623047
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/05/1B.GPM.GMI.TB2016.20170105-S093109-E110342.016223.V05A.HDF5
14.877037048339844
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/05/1B.GPM.GMI.TB2016.20170105-S201907-E215140.016230.V05A.HDF5
15.594902992248535
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/05/1B.GPM.GMI.TB2016.20170105-S140851-E154124.016226.V05A.HDF5
15.782471656799316
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/05/1B.GPM.GMI.TB2016.20170105-S184633-E201906.016229.V05A.HDF5
13.983699798583984
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/05/1B.GPM.GMI.TB2016.20170105-S215141-E232414.016231.V05A.HDF5
15.545907974243164
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/05/1B.G

13.652364730834961
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/09/1B.GPM.GMI.TB2016.20170109-S073741-E091015.016284.V05A.HDF5
16.298049926757812
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/09/1B.GPM.GMI.TB2016.20170109-S060507-E073740.016283.V05A.HDF5
13.583117485046387
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/09/1B.GPM.GMI.TB2016.20170109-S091016-E104248.016285.V05A.HDF5
8.888471603393555
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/09/1B.GPM.GMI.TB2016.20170109-S030000-E043233.016281.V05A.HDF5
14.45724105834961
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/09/1B.GPM.GMI.TB2016.20170109-S043234-E060506.016282.V05A.HDF5
13.355283737182617
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/09/1B.GPM.GMI.TB2016.20170109-S121523-E134756.016287.V05A.HDF5
15.945077896118164
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/09/1B.GPM.GMI.TB2016.20170109-S104249-E121522.016286.V05A.HDF5
14.210566520690918
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/09/1B.GPM

/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/13/1B.GPM.GMI.TB2016.20170113-S023856-E041129.016343.V05A.HDF5
14.414463996887207
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/13/1B.GPM.GMI.TB2016.20170113-S041130-E054403.016344.V05A.HDF5
14.023124694824219
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/13/1B.GPM.GMI.TB2016.20170113-S010623-E023855.016342.V05A.HDF5
16.182064056396484
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/13/1B.GPM.GMI.TB2016.20170113-S102145-E115418.016348.V05A.HDF5
15.151420593261719
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/13/1B.GPM.GMI.TB2016.20170113-S054404-E071637.016345.V05A.HDF5
13.987160682678223
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/13/1B.GPM.GMI.TB2016.20170113-S084912-E102144.016347.V05A.HDF5
13.089839935302734
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/13/1B.GPM.GMI.TB2016.20170113-S115419-E132652.016349.V05A.HDF5
17.494426727294922
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/13/1B.GPM.GMI.TB2016.20170

12.459972381591797
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/17/1B.GPM.GMI.TB2016.20170117-S082800-E100032.016409.V05A.HDF5
15.1840181350708
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/17/1B.GPM.GMI.TB2016.20170117-S235336-E012608.016419.V05A.HDF5
17.149105072021484
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/17/1B.GPM.GMI.TB2016.20170117-S021745-E035018.016405.V05A.HDF5
12.197161674499512
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/17/1B.GPM.GMI.TB2016.20170117-S065526-E082759.016408.V05A.HDF5
16.14603042602539
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/17/1B.GPM.GMI.TB2016.20170117-S035019-E052252.016406.V05A.HDF5
13.8574800491333
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/17/1B.GPM.GMI.TB2016.20170117-S100033-E113306.016410.V05A.HDF5
14.142557144165039
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/17/1B.GPM.GMI.TB2016.20170117-S113307-E130540.016411.V05A.HDF5
14.971287727355957
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/17/1B.GPM.GM

14.843565940856934
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/21/1B.GPM.GMI.TB2016.20170121-S215940-E233212.016480.V05A.HDF5
17.69852638244629
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/21/1B.GPM.GMI.TB2016.20170121-S185433-E202705.016478.V05A.HDF5
16.286203384399414
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/21/1B.GPM.GMI.TB2016.20170121-S202706-E215939.016479.V05A.HDF5
16.53263282775879
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/21/1B.GPM.GMI.TB2016.20170121-S032859-E050131.016468.V05A.HDF5
15.819801330566406
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/21/1B.GPM.GMI.TB2016.20170121-S015625-E032858.016467.V05A.HDF5
14.871901512145996
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/21/1B.GPM.GMI.TB2016.20170121-S002352-E015624.016466.V05A.HDF5
16.426706314086914
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/21/1B.GPM.GMI.TB2016.20170121-S050132-E063405.016469.V05A.HDF5
14.772768020629883
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/21/1B.GPM

17.047513961791992
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/25/1B.GPM.GMI.TB2016.20170125-S000219-E013451.016528.V05A.HDF5
16.24549102783203
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/25/1B.GPM.GMI.TB2016.20170125-S105012-E122244.016535.V05A.HDF5
14.866783142089844
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/25/1B.GPM.GMI.TB2016.20170125-S231037-E004310.016543.V05A.HDF5
17.277292251586914
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/25/1B.GPM.GMI.TB2016.20170125-S091739-E105011.016534.V05A.HDF5
17.392454147338867
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/25/1B.GPM.GMI.TB2016.20170125-S043959-E061231.016531.V05A.HDF5
15.538763046264648
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/25/1B.GPM.GMI.TB2016.20170125-S135518-E152750.016537.V05A.HDF5
14.420394897460938
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/25/1B.GPM.GMI.TB2016.20170125-S183258-E200530.016540.V05A.HDF5
13.905241966247559
/home/inderpreet/Dendrite/SatData/GMI/L1B/2017/01/25/1B.GP

In [12]:
d5 = xarray.Dataset({
    "y5": (["pixels", "scans",], y5),
})
outfile = os.path.expanduser("~/Dendrite/UserAreas/Kaur/2017_y5.nc")
d5.to_netcdf(outfile, mode = "w")

In [14]:
d95 = xarray.Dataset({
    "y95": (["pixels", "scans",], y95),
})
outfile = os.path.expanduser("~/Dendrite/UserAreas/Kaur/2017_y95.nc")
d95.to_netcdf(outfile, mode = "w")

In [ ]:
gmi_s    = GMI_Sat(gmifiles[:1])  

validation_data    = gmiSatData(gmi_s, 
                         inputs, outputs,
                         batch_size = batchSize,
                         latlims = latlims,
                         normalize = norm,      )

In [ ]:
inputs             = np.array(["ta", "t2m",  "wvp", "z0", "stype"])

In [ ]:
bins = np.arange(100, 300, 1)
plt.hist(training_data.x[:, 1],bins,  histtype = "step", density = True)
plt.hist(validation_data.x[:, :, 1].ravel(), bins, histtype = "step", density = True)


In [ ]:
fig, ax = plt.subplots(1, 1, figsize = [6, 6])
rndinds = np.random.randint(1, 2700, 100)
for i in rndinds:
    ax.plot(quantiles, y_pre[i, 10, :], 'b', alpha = 0.2)
ax.set_xlabel("quantiles")
ax.set_ylabel("IWP")
fig.savefig("quantiles.png")

In [ ]:
bins = np.array([0.0,.0001,.00025,.0005,0.001,.0025,.005,.01,.025,.05,.1,.25,.5,1,2, 5, 10, 20, 50])
  
fig, ax = plt.subplots(1, 1, figsize = [8, 8])
ax.hist(y_pos_mean.ravel(), bins = bins , density = True)
ax.set_yscale('log')
ax.set_xscale("log")

In [ ]:
import pickle
IWP  = np.concatenate(IWP, axis = 0)
LAT  = np.concatenate(LAT, axis = 0)
LON  = np.concatenate(LON, axis = 0)
LSM  = np.concatenate(LSM, axis = 0)
IWP0 = np.concatenate(IWP0, axis = 0)
IWP_mean = np.concatenate(IWP_mean, axis = 0)
with open("jan2020_IWP.pickle", "wb") as f:
    pickle.dump(IWP, f)
    pickle.dump(IWP0, f)
    pickle.dump(IWP_mean, f)
    pickle.dump(LON, f)
    pickle.dump(LAT, f)
    pickle.dump(LSM, f)
    
    
    f.close()